In [9]:
import pandas as pd
import json
for data_name in ['company']:
    for mode in ['rate', 'count']:
        graph_data = {}
        
        data = pd.read_parquet(f'../../dataset/{mode}/{data_name}.parquet')
        df = pd.read_parquet(f'../../dataset/graph/{data_name}.parquet')

        data['node_id'] = data.apply(lambda x: '-'.join([str(int(x[f"{i}_id"])) for i in data_name.split('-') + ['skill']]), axis=1)
        data = data.drop([f'{i}_id' for i in data_name.split('-') + ['skill']], axis=1)
        data = data.set_index(['node_id'])
        FX = data.values.T.tolist()
        graph_data['FX'] = FX

        data = data.reset_index().reset_index()
        graph_data['node_ids'] = {i:v for i,v in zip(data['node_id'], data['index'])}

        df['row_id'] = df.apply(lambda x: '-'.join([str(x[f"{i}_id"]) for i in data_name.split('-') + ['row']]), axis=1)
        df['col_id'] = df.apply(lambda x: '-'.join([str(x[f"{i}_id"]) for i in data_name.split('-') + ['col']]), axis=1)
        df = df[['row_id', 'col_id']]
        df['row_id'] = df['row_id'].apply(lambda x: graph_data['node_ids'][x])
        df['col_id'] = df['col_id'].apply(lambda x: graph_data['node_ids'][x])
        graph_data['edges'] = df.values.tolist()

        json.dump(graph_data, open(f'data/{mode}/{data_name}.json', 'w'))